In [163]:
%pip install pandas
%pip install ginza
%pip install ja_ginza_electra
%pip install ja_ginza
%pip install spacy
%pip install IPython

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
Note: you may need to restart the kernel to use updated packages.
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Home

In [3]:
# Import libraries
import pandas as pd # type: ignore
import glob
import sys
import re
import ginza
import spacy
import os

sys.path.append('..')

from src.utils.cleanText import clean_text
nlp = spacy.load('ja_ginza_electra')

# Set display options
pd.set_option('display.max_columns', 500)

/usr/local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/local/lib/python3.9/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
# read txt news file
news_file = '../data/news/news.txt'
# Get filename from the news_file path
filename = news_file.split('/')[-1]
# Read the news file
news_txt = open(news_file, 'r').read()
# Clean the text
news_txt = clean_text(news_txt)
# Split the text into sentences \n
news_txt = news_txt.split('\n')
print(news_txt)
# remove if there are empty strings ""
news_txt = [sentence for sentence in news_txt if sentence]

# Get the average length of the sentences in news_txt.
sentence_lengths = [len(sentence) for sentence in news_txt]
print(sentence_lengths)
average_sentence_length = sum(sentence_lengths)/len(sentence_lengths)

# Print results
print(f'The average length of the sentences in {filename} is {average_sentence_length} characters')




['朝鮮半島、台湾海峡、フィリピン周辺海域。', 'アジア太平洋地域には数多くの危機が伏在するからこそ、対話と交流を通じた信頼醸成は欠かせない。', 'これを長期的に安定した秩序の実現へつなげていくさらなる努力も必要だ。', '地域の防衛担当閣僚や専門家がシンガポールに集まって意見を交わす「アジア安全保障会議」が開かれた。', '会議に合わせて米中の国防相が、１年半ぶりに対面で会談した。', '台湾や南シナ海情勢で軍事緊張が高まる両国だけに激しい応酬になったが、危機時の相互連絡を話し合う作業部会を設けることで合意したのは一歩前進だ。', '特に米国では選挙をにらんで与野党が対中強硬姿勢を競う構図が強まる。', '不測の事態を防ぐだけでなく、こうした政治の動向からも距離を置いた冷静な対応を、双方の軍事当局には望みたい。', 'もっとも、これは最低限の目標と言うべきだろう。', 'アジアのみならず世界の秩序に重い責任を持つ米中両大国に本来求められるのは、中長期的な安保環境の改善だ。', '懸念されるのは、核戦力の増強とミサイル開発が地域で急速に進んでいることだ。', '米中の核兵器をめぐる交渉は昨年１１月以降、伝えられていない。', 'ロシアが無責任な核の脅しを仕掛けている折、米中は核の脅威の低減に向けて協働してほしい。', '中国の董軍国防相は会議での演説で、台湾海峡や南シナ海の緊張について「外部勢力による共謀や支援がある」と批判した。', '名指しは避けたものの米国の関与が問題だと言いたかったのだろう。', 'だが中国軍が南シナ海で岩礁を軍事拠点化するなど海空で存在感を強め、域内を緊張させているのは動かしがたい事実だ。', '会議出席者から言行不一致を指摘する声が相次いだことを、中国は重く受け止めねばならない。', '片や米国も、日本を含む同盟・友好国との対中抑止網の構築に余念がない。', '日本もフィリピン支援を通じて南シナ海への関与を強めている。', 'この状況にプラボウォ・インドネシア次期大統領が「地政学的な緊張の高まりにグローバルサウス（新興国、途上国）は幻滅している」と苦言を呈した。', '軍事力を競い合うのではなく外交を通じた安定実現を望む国々の声に、真摯（しんし）に耳を傾けるべきだ。', 'レーダー照射問題をめぐって日韓が再発防止策と防衛交流再開で合意

In [5]:
def analyze_ginza(text: str):
    ret = {}
    doc = nlp(text)
    for sentence in doc.sents:
        for token in ginza.bunsetu_head_tokens(sentence):
            for _relation, sub_phrase in ginza.sub_phrases(token):
                ret[str(sub_phrase)] = str(token)
    return dict(ret)

analyze_ginza("毎日焼肉定食を食べます")

{'毎日': '食べ', '焼肉+定食': '食べ'}

In [6]:
from collections import defaultdict
frames = defaultdict(lambda: 0)
sentences = set()

def tokenize(input_file: str):
    print(input_file)
    with open(input_file, 'r') as fin:
      for line in fin:
          try:
            doc = nlp(line.rstrip())
          except:
            continue
          for sent in doc.sents:
            if sent.text in sentences:
              continue
            sentences.add(sent.text)
            for t in ginza.bunsetu_head_tokens(sent):
              if t.pos_ not in {"ADJ", "VERB"}:
                continue
              v = ginza.phrase(ginza.lemma_)(t)
              dep_phrases = ginza.sub_phrases(t, ginza.phrase(ginza.lemma_), ginza.is_not_stop)
              subj = [phrase for dep, phrase in dep_phrases if dep in {"nsubj"}]
              obj  = [phrase for dep, phrase in dep_phrases if dep in {"obj", "iobj"}]
              for s in subj:
                for o in obj:
                  frames[(s, o, v)] += 1

    for frame, count in sorted(frames.items(), key=lambda t: -t[1]):
      print(count, *frame, sep="\t")

if __name__ == "__main__":
    # Input file
    input_file = '../data/news/news.txt'
    # Tokenize the input file
    tokenize(input_file)

../data/news/news.txt
1	与野党	対中+強硬+姿勢	競う
1	ロシア	脅し	仕掛ける
1	日本	フィリピン+支援	強める
1	日本	関与	強める
1	プラボウォ+・+インドネシア+次期+大統領	苦言	呈する


In [7]:
# Load newspaper data example json file
news = pd.read_json('../data/news/news.json')
news_lemmas = news['lemma']

print(news_lemmas.head(5))

0         が
1        外交
2        ぶり
3    インドネシア
4    シンガポール
Name: lemma, dtype: object


In [58]:
def read_json_files(path: str):
    files = sorted(glob.glob(path))
    dataframes = [pd.read_json(file) for file in files]
    combined_dataframe = pd.concat(dataframes, ignore_index=True)
    return combined_dataframe

def read_csv_files(path: str):
    files = sorted(glob.glob(path))
    dataframes = [pd.read_csv(file) for file in files]
    combined_dataframe = pd.concat(dataframes, ignore_index=True)
    return combined_dataframe

def match_tokens(input_dataframe, token_dataframes):
    # Creamos una nueva columna en el dataframe de entrada que es la concatenación de 'lemma' y 'pos'
    input_dataframe['lemma_pos'] = input_dataframe['lemma'] + '_' + input_dataframe['pos']
    # Convertimos los lemas del dataframe de entrada a un conjunto (eliminando duplicados)
    input_lemmas_pos = set(input_dataframe['lemma_pos'].dropna())
    matching_results = {}
    total_matching_lemmas_pos = 0
    # Iteramos sobre cada dataframe en token_dataframes
    for file_name, df in token_dataframes.items():
        # Creamos una nueva columna en el dataframe actual que es la concatenación de 'lemma' y 'pos'
        df['lemma_pos'] = df['lemma'] + '_' + df['pos']
        # Convertimos los lemas del dataframe actual a un conjunto (eliminando duplicados)
        token_lemmas_pos = set(df['lemma_pos'].dropna())
        # Encontramos la intersección de los dos conjuntos (lemas que están en ambos)
        matching_lemmas_pos = input_lemmas_pos & token_lemmas_pos
        num_matching_lemmas_pos = len(matching_lemmas_pos)
        total_matching_lemmas_pos += num_matching_lemmas_pos
        # Calculamos el porcentaje de lemas coincidentes
        percentage_matching_lemmas_pos = (num_matching_lemmas_pos / len(input_lemmas_pos)) * 100
        matching_results[file_name] = (num_matching_lemmas_pos, percentage_matching_lemmas_pos)
        print(f'{file_name}')
        print(f'\tTotal matching: {num_matching_lemmas_pos} - {percentage_matching_lemmas_pos:.2f}%')
        # Eliminamos los lemas coincidentes de input_lemmas para no contarlos más de una vez
        input_lemmas_pos -= matching_lemmas_pos
    print(f'Total matching: {total_matching_lemmas_pos} - {(total_matching_lemmas_pos / len(input_lemmas_pos)) * 100:.2f}%')

def match_lemmas(input_dataframe, vocabulary_dataframes):
    input_lemmas = input_dataframe['lemma']
    total_matching_hiragana = 0
    total_matching_kanji = 0
    for file_name, df in vocabulary_dataframes.items():
        matching_kanji = []
        matching_hiragana = []
        kanji_lemmas = df['Kanji'].dropna()
        hiragana_lemmas = df['Hiragana'].dropna()
        matching_kanji.extend(input_lemmas[input_lemmas.isin(kanji_lemmas)])
        num_matching_kanji = len(matching_kanji)
        total_matching_kanji += num_matching_kanji
        matching_hiragana.extend(input_lemmas[input_lemmas.isin(hiragana_lemmas)])
        num_matching_hiragana = len(matching_hiragana)
        total_matching_hiragana += num_matching_hiragana
        percentage_matching_kanji = (num_matching_kanji / len(input_lemmas)) * 100
        percentage_matching_hiragana = (num_matching_hiragana / len(input_lemmas)) * 100
        print(f'{file_name}')
        print(f'\tTotal matching Kanji: {num_matching_kanji} - {percentage_matching_kanji:.2f}%')
        print(f'\tTotal matching Hiragana: {num_matching_hiragana} - {percentage_matching_hiragana:.2f}%')
        print(f'\tTotal cumulative matching: {num_matching_kanji + num_matching_hiragana} - {(percentage_matching_kanji + percentage_matching_hiragana):.2f}%')
        print('\n')
    print(f'Total matching Kanji: {total_matching_kanji} - {(total_matching_kanji / len(input_lemmas)) * 100:.2f}%')
    print(f'Total matching Hiragana: {total_matching_hiragana} - {(total_matching_hiragana / len(input_lemmas)) * 100:.2f}%')

    return matching_kanji, matching_hiragana

input_token_dataframe = read_json_files('../data/news/*.json')
for file_name, df in input_token_dataframe.items():
    match_tokens(df, token_dataframes)
    print('\n-----------------------------------\n')
    match_lemmas(df, vocabulary_dataframes)



NameError: name 'token_dataframes' is not defined

In [10]:
# Get a list of all token json files in ../data/tokens ordered by filename
token_files = sorted(glob.glob('../data/tokens/*.json'))
total_matching_lemmas = 0
total_lemmas = 0

# Read all the json files in ../data/tokens
for token_file in token_files:
    # Read the json file
    token_data = pd.read_json(token_file)
    # Get the lemma column
    token_lemma = token_data['lemma']
    # Chek if the news_lemma is in the token_lemma
    matching_lemmas = token_lemma[token_lemma.isin(news_lemmas)]
    # Get the number of matching lemmas
    num_matching_lemmas = matching_lemmas.size
    # Get percentage of matching lemmas
    percentage_matching_lemmas = num_matching_lemmas / token_lemma.size
    # Increase the total number of matching lemmas
    total_matching_lemmas += num_matching_lemmas
    # Increase the total number of lemmas
    total_lemmas += token_lemma.size
    # Print the results
    print(f'{token_file}: {num_matching_lemmas} matching lemmas ({percentage_matching_lemmas:.2%})')
    
# Calculate the cumulative percentage of matching lemmas
cumulative_percentage_matching_lemmas = total_matching_lemmas / total_lemmas

print(len(news_lemmas))

# Print the cumulative percentage of matching lemmas
print(f'Total matching lemmas: {total_matching_lemmas}')
print(f'Total lemmas: {total_lemmas}')
print(f'Cumulative percentage of matching lemmas: {cumulative_percentage_matching_lemmas:.2%}')

# Get a list of all vocabulary files in ../data/ sorted by filename
vocabulary_files = sorted(glob.glob('../data/vocabulary/*.csv'))
total_matching_kanji = 0
total_matching_hiragana = 0
total_news_lemmas = news_lemmas.size

# Read all the csv files in ../data/vocabulary
for vocabulary_file in vocabulary_files:
    # Get fileName
    fileName = vocabulary_file.split('/')[-1]
    # Read the csv file
    vocabulary_df = pd.read_csv(vocabulary_file)
    # Get the kanji column
    kanji_list = vocabulary_df['Kanji']
    # Get the hiragana column
    hiragana_list = vocabulary_df['Hiragana']
    # Compare news_lemmas with kanji_list
    matching_kanji = news_lemmas[news_lemmas.isin(kanji_list)]
    # Get the number of matching kanji
    num_matching_kanji = len(matching_kanji)
    # Get the percentage of matching kanji
    percentage_matching_kanji = (num_matching_kanji / len(kanji_list)) * 100
    # Increase the total number of matching kanji
    total_matching_kanji += num_matching_kanji

    # Compare news_lemmas with hiragana_list
    matching_hiragana = news_lemmas[news_lemmas.isin(hiragana_list)]
    # Get the number of matching hiragana
    num_matching_hiragana = len(matching_hiragana)
    # Get the percentage of matching hiragana
    percentage_matching_hiragana = (num_matching_hiragana / len(hiragana_list)) * 100
    # Increase the total number of matching hiragana
    total_matching_hiragana += num_matching_hiragana

    # Print the results by vocabulary file
    print(f'Total matching kanji in {fileName}: {num_matching_kanji} - {percentage_matching_kanji:.2f}%')
    print(f'Total matching hiragana in {fileName}: {num_matching_hiragana} - {percentage_matching_hiragana:.2f}%')

# Calculate the cumulative percentage of matching kanji
cumulative_percentage_matching_kanji = total_matching_kanji / total_news_lemmas
cumulative_percentage_matching_hiragana = total_matching_hiragana / total_news_lemmas

# Print the cumulative percentage of matching kanji
print(f'Total matching kanji: {total_matching_kanji}')
print(f'Cumulative percentage of matching kanji: {cumulative_percentage_matching_kanji:.2%}')
print(f'Total matching hiragana: {total_matching_hiragana}')
print(f'Cumulative percentage of matching hiragana: {cumulative_percentage_matching_hiragana:.2%}')

# Save matching kanji to a csv file
matching_kanji_df = pd.DataFrame(matching_kanji)
matching_kanji_df.to_csv('../data/matching_kanji.csv', index=False)

../data/tokens/lvl0Tokens.json: 205 matching lemmas (8.39%)
../data/tokens/lvl1Tokens.json: 173 matching lemmas (9.77%)
../data/tokens/lvl2Tokens.json: 214 matching lemmas (8.98%)
../data/tokens/lvl3Tokens.json: 213 matching lemmas (8.74%)
../data/tokens/lvl4Tokens.json: 194 matching lemmas (7.95%)
295
Total matching lemmas: 999
Total lemmas: 11474
Cumulative percentage of matching lemmas: 8.71%
Total matching kanji in n1vocab.csv: 43 - 1.24%
Total matching hiragana in n1vocab.csv: 22 - 0.63%
Total matching kanji in n2vocab.csv: 11 - 0.60%
Total matching hiragana in n2vocab.csv: 22 - 1.20%
Total matching kanji in n3vocab.csv: 49 - 2.67%
Total matching hiragana in n3vocab.csv: 29 - 1.58%
Total matching kanji in n4vocab.csv: 13 - 2.05%
Total matching hiragana in n4vocab.csv: 11 - 1.74%
Total matching kanji in n5vocab.csv: 17 - 2.54%
Total matching hiragana in n5vocab.csv: 23 - 3.44%
Total matching kanji: 133
Cumulative percentage of matching kanji: 45.08%
Total matching hiragana: 107
Cum

In [32]:
# Get a list of all vocabulary files in ../data/ sorted by filename
vocabulary_files = sorted(glob.glob('../data/vocabulary/*.csv'))
total_matching_kanji = 0
total_matching_hiragana = 0
total_news_lemmas = news_lemmas.size

# Read all the csv files in ../data/vocabulary
for vocabulary_file in vocabulary_files:
    # Get fileName
    fileName = vocabulary_file.split('/')[-1]
    # Read the csv file
    vocabulary_df = pd.read_csv(vocabulary_file)
    # Get the kanji column
    kanji_list = vocabulary_df['Kanji']
    # Get the hiragana column
    hiragana_list = vocabulary_df['Hiragana']
    # Compare news_lemmas with kanji_list
    matching_kanji = news_lemmas[news_lemmas.isin(kanji_list)]
    # Get the number of matching kanji
    num_matching_kanji = len(matching_kanji)
    # Get the percentage of matching kanji
    percentage_matching_kanji = (num_matching_kanji / len(kanji_list)) * 100
    # Increase the total number of matching kanji
    total_matching_kanji += num_matching_kanji

    # Compare news_lemmas with hiragana_list
    matching_hiragana = news_lemmas[news_lemmas.isin(hiragana_list)]
    # Get the number of matching hiragana
    num_matching_hiragana = len(matching_hiragana)
    # Get the percentage of matching hiragana
    percentage_matching_hiragana = (num_matching_hiragana / len(hiragana_list)) * 100
    # Increase the total number of matching hiragana
    total_matching_hiragana += num_matching_hiragana

    # Print the results by vocabulary file
    print(f'Total matching kanji in {fileName}: {num_matching_kanji} - {percentage_matching_kanji:.2f}%')
    print(f'Total matching hiragana in {fileName}: {num_matching_hiragana} - {percentage_matching_hiragana:.2f}%')

# Calculate the cumulative percentage of matching kanji
cumulative_percentage_matching_kanji = total_matching_kanji / total_news_lemmas
cumulative_percentage_matching_hiragana = total_matching_hiragana / total_news_lemmas

# Print the cumulative percentage of matching kanji
print(f'Total matching kanji: {total_matching_kanji}')
print(f'Cumulative percentage of matching kanji: {cumulative_percentage_matching_kanji:.2%}')
print(f'Total matching hiragana: {total_matching_hiragana}')
print(f'Cumulative percentage of matching hiragana: {cumulative_percentage_matching_hiragana:.2%}')

# Save matching kanji to a csv file
matching_kanji_df = pd.DataFrame(matching_kanji)
matching_kanji_df.to_csv('../data/matching_kanji.csv', index=False)


    # # Compare news_lemmas with kanji_list
    # matching_kanji = news_lemmas[news_lemmas.isin(kanji_list)]
    # print(f'filename {fileName} matching_kanji {matching_kanji}')
    # # Get the number of matching kanji
    # num_matching_kanji = len(matching_kanji)
    # # Get the percentage of matching kanji
    # percentage_matching_kanji = (num_matching_kanji / len(kanji_list)) * 100
    # # Increase the total number of matching kanji
    # total_matching_kanji += num_matching_kanji

#     # Get the hiragana column
#     hiragana_list = vocabulary_df['Hiragana']
#     # Compare news_lemmas with hiragana_list
#     matching_hiragana = news_lemmas[news_lemmas.isin(hiragana_list)]
#     # Get the number of matching hiragana
#     num_matching_hiragana = len(matching_hiragana)
#     # Get the percentage of matching hiragana
#     percentage_matching_hiragana = (num_matching_hiragana / len(hiragana_list)) * 100
#     # Increase the total number of matching hiragana
#     total_matching_hiragana += num_matching_hiragana

#     # Print the results by vocabulary file
#     print(f'Total matching kanji in {fileName}: {num_matching_kanji} - {percentage_matching_kanji:.2f}%')
#     print(f'Total matching hiragana in {fileName}: {num_matching_hiragana} - {percentage_matching_hiragana:.2f}%')

# # Calculate the cumulative percentage of matching kanji
# cumulative_percentage_matching_kanji = total_matching_kanji / total_news_lemmas
# cumulative_percentage_matching_hiragana = total_matching_hiragana / total_news_lemmas

# # Print the cumulative percentage of matching kanji
# print(f'Total matching kanji: {total_matching_kanji}')
# print(f'Cumulative percentage of matching kanji: {cumulative_percentage_matching_kanji:.2%}')
# print(f'Total matching hiragana: {total_matching_hiragana}')
# print(f'Cumulative percentage of matching hiragana: {cumulative_percentage_matching_hiragana:.2%}')

# Save matching kanji to a csv file
# matching_kanji_df = pd.DataFrame(matching_kanji)
# matching_kanji_df.to_csv('../data/matching_kanji.csv', index=False)



Total matching kanji in n1vocab.csv: 43 - 1.24%
Total matching hiragana in n1vocab.csv: 22 - 0.63%
Total matching kanji in n2vocab.csv: 11 - 0.60%
Total matching hiragana in n2vocab.csv: 22 - 1.20%
Total matching kanji in n3vocab.csv: 49 - 2.67%
Total matching hiragana in n3vocab.csv: 29 - 1.58%
Total matching kanji in n4vocab.csv: 13 - 2.05%
Total matching hiragana in n4vocab.csv: 11 - 1.74%
Total matching kanji in n5vocab.csv: 17 - 2.54%
Total matching hiragana in n5vocab.csv: 23 - 3.44%
Total matching kanji: 133
Cumulative percentage of matching kanji: 45.08%
Total matching hiragana: 107
Cumulative percentage of matching hiragana: 36.27%


In [61]:
from IPython.display import display

def match_vocabulary(input_dataframe, vocabulary_dataframes, language_level):
    matching_tokens = {}

    # Define the language levels in order
    language_levels = ['N1', 'N2', 'N3', 'N4', 'N5']

    for file_name, vocabulary_df in vocabulary_dataframes.items():
        # Get Level from file_name the filename is in the two first characters
        level = file_name[:2].upper()

        # If the level is higher than the language_level, skip the dataframe
        if language_levels.index(level) < language_levels.index(language_level.upper()):

            # Get the kanji column
            kanji_list = vocabulary_df['Kanji']
            # Get the hiragana column
            hiragana_list = vocabulary_df['Hiragana']
            # Get the input_dataframe lemma column
            input_lemmas = input_dataframe['lemma']

            # Find matching rows in the input dataframe
            matching_rows = input_dataframe[
                input_lemmas[input_lemmas.isin(kanji_list)] | 
                input_lemmas[input_lemmas.isin(hiragana_list)]
            ]

            # 
            if not matching_rows.empty:
                matching_tokens[level] = matching_rows

    # Concatenate all matching tokens into one dataframe
    if matching_tokens:
        coincidences_df = pd.concat(matching_tokens.values())
        # Save the coincidences_df to a csv file
        # coincidences_df.to_csv('../data/coincidences.csv', index=False)   
        display(coincidences_df)
    else:
        print('No matching tokens found')

if __name__ == "__main__":
    # Match the tokens
    input_token_dataframe = read_json_files('../data/news/*.json')
    vocaabulary_dataframes = read_csv_files('../data/vocabulary/*.csv')

    match_vocabulary(input_token_dataframe, vocabulary_dataframes, 'N2')

TypeError: unsupported operand type(s) for |: 'str' and 'bool'

In [110]:
def match_vocabulary(input_dataframe, vocabulary_dataframes, language_level):
    # Set display options to show all rows
    pd.set_option('display.max_rows', None)

    matching_tokens = {}

    # Define the language levels in order
    language_levels = ['N5', 'N4', 'N3', 'N2', 'N1']

    for filename, vocabulary_df in vocabulary_dataframes.items():
        # Get Level from file_name the filename is in the two first characters
        level = filename[:2].upper()

        # Get the input_dataframe lemma column
        input_lemmas = input_dataframe['lemma']

        if language_levels.index(level) > language_levels.index(language_level.upper()):
            print(f'languague level {level} is higher than {language_level.upper()}')
            # Get the kanji column
            kanji_list = vocabulary_df['Kanji']
            # Get the hiragana column
            hiragana_list = vocabulary_df['Hiragana']

            # If input_lemmas contains any of the kanji_list or hiragana_list save all the row in a new dataframe
            matching_kanji = input_lemmas[input_lemmas.isin(kanji_list)]
            matching_hiragana = input_lemmas[input_lemmas.isin(hiragana_list)]

            matching_rows = vocabulary_df[
                (vocabulary_df['Kanji'].isin(matching_kanji)) |
                (vocabulary_df['Hiragana'].isin(matching_hiragana))
            ]

            if not matching_rows.empty:
                matching_tokens[level] = matching_rows

    # Display each level matching tokens
    if matching_tokens:
        for level, matching_tokens_df in matching_tokens.items():
            print(f'{level} vocabulary:')
            print(matching_tokens_df.to_string(index=False))  # Print DataFrame without index


if __name__ == "__main__":
    # Match the tokens
    input_token_dataframe = read_json_files('../data/news/*.json')
    vocaabulary_dataframes = read_csv_files('../data/vocabulary/*.csv')

    match_vocabulary(input_token_dataframe, vocabulary_dataframes, 'N3')

languague level N1 is higher than N3
languague level N2 is higher than N3
N1 vocabulary:
Kanji Hiragana                                                                                                      English
    相       あい                                                                                   together, mutually, fellow
  集まる     あつまる                                                                           to gather, to collect, to assemble
    網       あみ                                                                                                 net, network
   或る       ある                                                                                        a certain..., some...
  NaN       いく                                                                                           to come, to orgasm
    軍      いくさ                                                                                 war, battle, campaign, fight
   入る       いる                             